In [1]:
import numpy as np
import pandas as pd
from urllib.parse import urlparse, parse_qsl
import html

import plotly.express as px
import scattertext as st

import re


c:\Users\tmros\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## SQL Injection Dataset Augmentation (Addressing Class Imbalance)

The SQL injection dataset obtained from Kaggle showed a noticeable imbalance between benign and malicious samples, which would negatively impact model training. To correct this, a hybrid augmentation strategy was used to strengthen and expand the SQLi class.

The final SQLi dataset combines:

1. Kaggle SQL Injection Dataset

Source: https://www.kaggle.com/datasets/sajid576/sql-injection-dataset

This dataset contains a large collection of handcrafted SQL injection strings covering common attack categories such as:

- authentication bypass
- UNION-based injections
- logical/boolean-based injections
- query manipulation and tautologies

However, despite its variety, the distribution of samples remained skewed, with SQLi payloads under-represented relative to benign traffic.

2. SQLMap-Generated SQLi Payloads (DVWA Environment)

To counter this imbalance, additional SQLi samples were generated using SQLMap against a DVWA (Damn Vulnerable Web Application) instance running on VirtualBox.
A custom capture script logged:

- parameter values modified by SQLMap
- injected SQLi payloads
- HTTP request metadata

These locally generated samples provide more realistic, execution-level SQL injection behaviour that complements the Kaggle dataset.

In [2]:
# Loading the dataset
data = pd.read_csv("C:\\Users\\tmros\\OneDrive\\Desktop\\Modifying Datasets\\final_sqli_dataset.csv")

#### Finding out the structure of the data

In [3]:
print(data.head())

# Shape of the dataset
print("Shape of the dataset:", data.shape)

                                               Query  Label
0  1 or extractvalue  (  1297,concat  (  0x5c,0x7...      1
1     SELECT sat, exact, classroom FROM clean WHE...      0
2  1' OR NOT ORD(MID((SELECT IFNULL(CAST(avatar A...      1
3                             SELECT * FROM behavior      0
4                                              47609      0
Shape of the dataset: (35555, 2)


In [4]:
data.describe(include=['O'])

,Query
count,35555
unique,35540
top,#NAME?
freq,8


In [5]:
data.describe()

,Label
count,35555.000000
mean,0.450513
std,0.497552
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


Thus, we deduce that there are less SQLIA queries than benign queries

In [6]:
# Drop rows where 'Query' is NaN
data = data.dropna(subset=['Query'])

# Then drop rows where 'Query' is empty or only whitespace
data = data[data['Query'].str.strip() != ""]

# Reset index
data = data.reset_index(drop=True)

# Confirm no empty queries remain
print("Remaining empty queries:", (data['Query'].str.strip() == "").sum())



Remaining empty queries: 0


### Checking if the data is balanced

In [7]:
data['Label'].value_counts()/data.shape[0] * 100

Label
0    54.950217
1    45.049783
Name: count, dtype: float64

The dataset is fairly balanced since out of 2 queries, one of them is likely to be SQLIA

In [8]:
data.isnull().sum()

Query    0
Label    0
dtype: int64

In [9]:
# Copying dataset to mod_data, where
# we will make the changes, keeping
# original dataframe 'data' as it is.
mod_data = data.copy()

In [10]:
# converting 'Label' to boolean column 'SQLIA'
# SQLIA = True -> Query is SQLIA
# SQLIA = False -> Query is not SQLIA
mod_data['SQLIA'] = mod_data['Label'].apply(bool)
mod_data.drop('Label', axis=1, inplace=True)

In [11]:
mod_data

,Query,SQLIA
0,"1 or extractvalue ( 1297,concat ( 0x5c,0x7...",True
1,"SELECT sat, exact, classroom FROM clean WHE...",False
2,1' OR NOT ORD(MID((SELECT IFNULL(CAST(avatar A...,True
3,SELECT * FROM behavior,False
4,47609,False
...,...,...
35549,update,True
35550,SELECT * FROM provide WHERE parallel = 'wore' ...,False
35551,UPDATE lower SET uncle = 'engineer'WHERE earn...,False
35552,1'+ ( select immd where 7499 = 7499 union ...,True


In [12]:
mod_data.duplicated().sum()

np.int64(13)

In [13]:
# removing duplicat rows
mod_data.drop_duplicates(inplace=True)
mod_data.shape

(35541, 2)

In [14]:
mod_data.to_csv('final_sqli_dataset.csv', index=False)